<a href="https://colab.research.google.com/github/la2015-hw/Group_10/blob/main/MLP_morphology_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [2]:
!pip install kaggle --quiet

In [3]:
# The following code will only execute
# successfully when compression is complete

import kagglehub

# Download latest version
path = kagglehub.dataset_download("mustansireranpurwala/sdss-image-dataset")

print("Path to dataset files:", path)

100%|██████████| 9.48G/9.48G [01:06<00:00, 152MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mustansireranpurwala/sdss-image-dataset/versions/7


In [4]:
import h5py
import numpy as np
import os

# Use the KaggleHub download path
import kagglehub
dataset_path = "/root/.cache/kagglehub/datasets/mustansireranpurwala/sdss-image-dataset/versions/7/processed_sdss_balanced.h5"


print("Path to dataset file:", dataset_path)

# Inspect file
with h5py.File(dataset_path, "r") as f:
    print("Keys in HDF5 file:", list(f.keys()))
    for key in f.keys():
        print(f"{key}: shape={f[key].shape}, dtype={f[key].dtype}")

# Extract features and labels
with h5py.File(dataset_path, "r") as f:
    # Target label
    y = np.array(f["broad_class"][:]).astype(str)

    # Features (all photometric + color + positional)
    X = np.column_stack([
        f["u_mag"][:],
        f["g_mag"][:],
        f["r_mag"][:],
        f["i_mag"][:],
        f["z_mag"][:],
        f["u_minus_g"][:],
        f["g_minus_r"][:],
        f["r_minus_i"][:],
        f["i_minus_z"][:],
        f["specz_redshift"][:],
        f["e_bv"][:],
        f["specz_redshift_err"][:],
        f["dec"][:],
        f["ra"][:]
    ])


Path to dataset file: /root/.cache/kagglehub/datasets/mustansireranpurwala/sdss-image-dataset/versions/7/processed_sdss_balanced.h5
Keys in HDF5 file: ['ObjID', 'broad_class', 'death_status', 'dec', 'e_bv', 'g_mag', 'g_minus_r', 'i_mag', 'i_minus_z', 'images', 'r_mag', 'r_minus_i', 'ra', 'specObjID', 'specz_redshift', 'specz_redshift_err', 'u_mag', 'u_minus_g', 'z_mag']
ObjID: shape=(47600,), dtype=|S20
broad_class: shape=(47600,), dtype=object
death_status: shape=(47600,), dtype=|S15
dec: shape=(47600,), dtype=float64
e_bv: shape=(47600,), dtype=float32
g_mag: shape=(47600,), dtype=float32
g_minus_r: shape=(47600,), dtype=float32
i_mag: shape=(47600,), dtype=float32
i_minus_z: shape=(47600,), dtype=float32
images: shape=(47600, 107, 107, 5), dtype=float32
r_mag: shape=(47600,), dtype=float32
r_minus_i: shape=(47600,), dtype=float32
ra: shape=(47600,), dtype=float64
specObjID: shape=(47600,), dtype=int64
specz_redshift: shape=(47600,), dtype=float32
specz_redshift_err: shape=(47600,), 

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print("Label classes:", label_encoder.classes_)

# Train-test split (80 % / 20 %)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# Normalize features for stable training
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Label classes: ['Elliptical' 'Irregular' 'Spiral-barred' 'Spiral-unbarred']


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# --- Architecture 1: 256-128-64 ---
def build_mlp_m1(input_dim, n_classes):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(n_classes, activation='softmax')
    ])
    return model


# --- Architecture 2: 512-256-128 ---
def build_mlp_m2(input_dim, n_classes):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(n_classes, activation='softmax')
    ])
    return model


# --- Architecture 3: 128-64 ---
def build_mlp_m3(input_dim, n_classes):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(n_classes, activation='softmax')
    ])
    return model


In [7]:
def run_experiment_morph(build_fn, X_train, y_train, lr=1e-4, epochs=30, run_name='morph_experiment'):

    n_classes = len(np.unique(y_train))
    model = build_fn(X_train.shape[1], n_classes)

    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1),
        ModelCheckpoint(f"{run_name}_best_morph.h5", monitor='val_loss', save_best_only=True)
    ]

    history = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=epochs,
        batch_size=128,
        callbacks=callbacks,
        verbose=1
    )

    # Best epoch
    best_epoch = int(np.argmax(history.history['val_accuracy']))

    print(f"\nBest epoch: {best_epoch+1}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

    return model, history


In [ ]:


model_m1a, hist_m1a = run_experiment_morph(
    build_mlp_m1,
    X_train, y_train, 1e-4,
    run_name='morph_mlp_256_128_64_1e-4'
)
model_m1b, hist_m1b = run_experiment_morph(
    build_mlp_m1,
    X_train, y_train, 1e-3,
    run_name='morph_mlp_256_128_64_1e-3'
)
model_m1c, hist_m1c = run_experiment_morph(
    build_mlp_m1,
    X_train, y_train, 5e-4,
    run_name='morph_mlp_256_128_64_5e-4'
)


model_m2a, hist_m2a = run_experiment_morph(
    build_mlp_m2,
    X_train, y_train, 1e-4,
    run_name='morph_mlp_512_256_128_1e-4'
)
model_m2b, hist_m2b = run_experiment_morph(
    build_mlp_m2,
    X_train, y_train, 1e-3,
    run_name='morph_mlp_512_256_128_1e-4'
)
model_m2c, hist_m2c = run_experiment_morph(
    build_mlp_m2,
    X_train, y_train, 5e-3,
    run_name='morph_mlp_512_256_128_1e-4'
)


model_m3a, hist_m3a = run_experiment_morph(
    build_mlp_m3,
    X_train, y_train, 1e-4,
    run_name='morph_mlp_128_64'
)
model_m3b, hist_m3b = run_experiment_morph(
    build_mlp_m3,
    X_train, y_train, 1e-3,
    run_name='morph_mlp_128_64'
)
model_m3c, hist_m3c = run_experiment_morph(
    build_mlp_m3,
    X_train, y_train, 5e-3,
    run_name='morph_mlp_128_64'
)




/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
265/268 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2886 - loss: 1.3888

268/268 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.2891 - loss: 1.3885 - val_accuracy: 0.4049 - val_loss: 1.3142 - learning_rate: 1.0000e-04
Epoch 2/30
265/268 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3782 - loss: 1.3261

268/268 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.3783 - loss: 1.3260 - val_accuracy: 0.4320 - val_loss: 1.2719 - learning_rate: 1.0000e-04
Epoch 3/30
259/268 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4046 - loss: 1.2953

268/268 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4047 - loss: 1.2951 - val_accuracy: 0.4407 - val_loss: 1.2492 - learning_rate: 1.0000e-04
Epoch 4/30
260/268 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4130 - loss: 1.2834

...

268/268 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.3928 - loss: 1.2969 - val_accuracy: 0.4488 - val_loss: 1.2216 - learning_rate: 0.0010
Epoch 2/30
265/268 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4461 - loss: 1.2343

268/268 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.4461 - loss: 1.2342 - val_accuracy: 0.4611 - val_loss: 1.2051 - learning_rate: 0.0010
Epoch 3/30
267/268 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4520 - loss: 1.2138

268/268 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4521 - loss: 1.2138 - val_accuracy: 0.4598 - val_loss: 1.1954 - learning_rate: 0.0010
Epoch 4/30
263/268 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4593 - loss: 1.2080

268/268 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4593 - loss: 1.2080 - val_accuracy: 0.4690 - val_loss: 1.1875 - learning_rate: 0.0010
Epoch 5/30
266/268 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4593 - loss: 1.2018

268/268 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4593 - loss: 1.2017 - val_accuracy: 0.4769 - val_loss: 1.1754 - learning_rate: 0.0010
Epoch 6/30
268/268 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4671 - loss: 1.1917 - val_accuracy: 0.4753 - val_loss: 1.1756 - learning_rate: 0.0010
Epoch 7/30
244/268 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4673 - loss: 1.1866